## SUMMARY: Will correct the h5 prediction files for all of the multi-instance videos for a specified color pair

# get color vids

In [2]:
import sys
parent_dir = os.path.dirname(current_dir)
current_dir = os.path.dirname(os.path.abspath(__file__))
sys.path.append(parent_dir)
sys.path.append('')

SyntaxError: invalid syntax (2636570829.py, line 1)

In [3]:
import os
import random
import h5py
import numpy as np
import time

from utils import find_node_velocity, get_stats, fill_missing, graph_vels, nan_vals

ModuleNotFoundError: No module named 'utils'

In [2]:
defaultdir = '/gpfs/radev/pi/saxena/aj764'
rootdir = f'{defaultdir}/PairedTestingSessions/'

Compiles a list of all of the sessions in PairedTestingSessions that have a Videos folder in them

In [3]:
vid_subdirs = []
for subdir, dirs, files in os.walk(rootdir):
    if subdir.endswith("Videos"):
        vid_subdirs.append(subdir)
vid_subdirs = sorted(vid_subdirs)

Seperates all of the videos into single instance videos and multi instance videos (and takes out videos from before April).

In [4]:
single_vids = {}
multi_vids = {}
for vids in vid_subdirs:
    files = os.listdir(vids)
    cut_vids = vids[28:]
    single_vids[cut_vids] = []
    multi_vids[cut_vids] = []
    for file in files:
        if file.endswith('.mp4') and int(file[:2]) >= 4:
            KL_count = file.count('KL')
            EB_count = file.count('EB')
            HF_count = file.count('HF')
            if KL_count + EB_count + HF_count == 2:
                multi_vids[cut_vids].append(file)
            else:
                single_vids[cut_vids].append(file)

In [5]:
single_len_tot = 0
for key, value in single_vids.items():
    single_len_tot += len(value)
print(f'There are {single_len_tot} single instance videos')

There are 1106 single instance videos


In [6]:
multi_len_tot = 0
for key, value in multi_vids.items():
    multi_len_tot += len(value)
print(f'There are {multi_len_tot} multi instance videos')

There are 905 multi instance videos


In [7]:
# split the multi-instance videos into their respective color pairs...
color_vids = {}
for key, value in multi_vids.items():
    for vid in value:
        trial_color = [vid[-12], vid[-5]]
        trial_key = ''
        if 'R' in trial_color:
            trial_key += 'R'
        if 'G' in trial_color:
            trial_key += 'G'
        if 'Y' in trial_color:
            trial_key += 'Y'
        if 'B' in trial_color:
            trial_key += 'B'
        if trial_key not in color_vids.keys():
            color_vids[trial_key] = []
        color_vids[trial_key].append(vid)

In [8]:
len_tot = 0
for key, value in color_vids.items():
    print(f'There are {len(value)} videos from {key} color pair')
    len_tot += len(value)
print('\n')
print(f'There are {len_tot} multi instance videos')

There are 128 videos from RY color pair
There are 169 videos from YB color pair
There are 168 videos from GB color pair
There are 176 videos from GY color pair
There are 99 videos from RB color pair
There are 165 videos from RG color pair


There are 905 multi instance videos


# fills in missing vals

In [9]:
CHECK = False
color_pair = 'YB'

In [35]:
start_time = time.time()

total_intial_nan = 0
total_after_out_nan = 0
total_final_nan = 0
bad_vids = []

for i, session in enumerate(multi_vids.keys()): 
    video_list = multi_vids[session]
    analysis_path = defaultdir + '/' + session[:-6] + 'Tracking/h5/'
    
    for video in video_list:
        if video in color_vids[color_pair]:
            # open analysis file
            analysis_file = analysis_path + video[:-3] + 'predictions.h5'
            with h5py.File(analysis_file,'r+') as f:
                locations = f["tracks"][:].T 
    
                # find nan values
                intial = nan_vals(locations)

                if intial > 20:
                    bad_vids.append(video)
    
                # just to check you haven't already done this vid or it isn't empty
                if intial != 0:
                    # take out positional outliers
                    for rat in range(locations.shape[-1]): # for each rat (not actually necessary, the dims work out without this loop but I don't feel like thinking abt that)
                        all_vels = {}
                        for node in range(locations.shape[1]): # for each node
                            # find the velocities
                            all_vels[node] = find_node_velocity(locations[:, node, :,  rat:rat+1])
                        
                            # get values need to find outliers
                            mean, std, low, high = get_stats(all_vels[node])
                        
                            # if you want to check that these values looks good
                            graph_vels(all_vels[node], CHECK)
                        
                            # replace outliers in locations with nan
                            nan_index = [i for i in range(len(all_vels[node])) if (all_vels[node][i] > high or all_vels[node][i] < low)]
                            for index in nan_index:
                                locations[index + 1, node, 0, rat], locations[index + 1, node, 0, rat] = np.nan, np.nan
                        
                            # if you want to check that new locations look good
                            test_vels = find_node_velocity(locations[:, node, :])
                            graph_vels(test_vels, check=CHECK, old_low=low, old_high=high)
        
                    # find nan values again
                    after_out = nan_vals(locations)
        
                    # fill in missing locations
                    print(f'video name: {video}')
                    new_locations = fill_missing(locations)
                    f["tracks"][:] = new_locations.T
    
                    # finds nan values for final time
                    after_fill = nan_vals(new_locations)
                    
                    total_intial_nan += intial
                    total_after_out_nan += after_out
                    total_final_nan += after_fill
        
                    # if you want to check the nan/fill values for a each video
                    if True:
                        # print(f'video name: {video}')
                        print(f'intial nan: {round(intial, 2)} %, after out nan: {round(after_out, 2)} %, final nan: {round(after_fill, 2)} %')
        
print('totals:')
print(f'intial nan: {round(total_intial_nan / len(color_vids[color_pair]), 2)} %, after out nan: {round(total_after_out_nan / len(color_vids[color_pair]), 2)} %, final nan: {round(total_final_nan / len(color_vids[color_pair]), 2)} %')
print(f'time elapse: {time.time() - start_time}')

video name: 041624_Cam4_TrNum6_Comp_KL001B-KL001Y.mp4
intial nan: 5.85 %, after out nan: 6.75 %, final nan: 0.0 %
video name: 041624_Cam4_TrNum12_Coop_KL001B-KL001Y.mp4
intial nan: 49.75 %, after out nan: 49.9 %, final nan: 0.0 %
video name: 041624_Cam4_TrNum10_Coop_KL001B-KL001Y.mp4
intial nan: 1.45 %, after out nan: 2.15 %, final nan: 0.0 %
video name: 041624_Cam4_TrNum8_Comp_KL001B-KL001Y.mp4
intial nan: 4.92 %, after out nan: 5.76 %, final nan: 0.0 %
video name: 041624_Cam3_TrNum5_Comp_KL002B-KL002Y.mp4
intial nan: 15.6 %, after out nan: 16.2 %, final nan: 0.0 %
video name: 041624_Cam3_TrNum7_Coop_KL002B-KL002Y.mp4
intial nan: 4.12 %, after out nan: 4.8 %, final nan: 0.0 %
video name: 041624_Cam3_TrNum12_Comp_KL002B-KL002Y.mp4
intial nan: 13.28 %, after out nan: 14.01 %, final nan: 0.0 %
video name: 041624_Cam3_TrNum9_Coop_KL002B-KL002Y.mp4
intial nan: 7.08 %, after out nan: 7.77 %, final nan: 0.0 %
video name: 041624_Cam2_TrNum7_Ineq_KL003B-KL003Y.mp4
intial nan: 9.41 %, after out

# check our work... (abt 33 vids that I WOULDN'T trust!!)

In [36]:
print(f'intial nan: {round(total_intial_nan / len(color_vids[color_pair]), 2)} %, after out nan: {round(total_after_out_nan / len(color_vids[color_pair]), 2)} %, final nan: {round(total_final_nan / len(color_vids[color_pair]), 2)} %')


intial nan: 13.85 %, after out nan: 14.44 %, final nan: 0.0 %


In [37]:
print(f'percent of videos intitially have over 1/5 of values nan: {round(100 * len(bad_vids) / len(color_vids[color_pair]) ,2)}% ')

percent of videos intitially have over 1/5 of values nan: 19.53% 


In [38]:
# the videos in question :(
bad_vids

['041624_Cam4_TrNum12_Coop_KL001B-KL001Y.mp4',
 '041924_Cam2_TrNum12_Coop_KL001B-KL001Y.mp4',
 '041924_Cam2_TrNum9_Coop_KL001B-KL001Y.mp4',
 '041924_Cam2_TrNum7_Comp_KL001B-KL001Y.mp4',
 '041924_Cam2_TrNum5_Comp_KL001B-KL001Y.mp4',
 '042324_Cam3_TrNum7_Coop_KL001B-KL001Y.mp4',
 '042324_Cam3_TrNum5_Comp_KL001B-KL001Y.mp4',
 '042324_Cam3_TrNum12_Coop_KL001B-KL001Y.mp4',
 '042324_Cam3_TrNum10_Comp_KL001B-KL001Y.mp4',
 '042624_Cam2_TrNum12_Comp_KL001B-KL001Y.mp4',
 '042624_Cam2_TrNum8_Coop_KL001B-KL001Y.mp4',
 '042624_Cam2_TrNum6_Coop_KL001B-KL001Y.mp4',
 '042624_Cam2_TrNum10_Comp_KL001B-KL001Y.mp4',
 '043024_Cam2_TrNum8_Coop_KL001B-KL001Y.mp4',
 '043024_Cam2_TrNum6_Coop_KL001B-KL001Y.mp4',
 '043024_Cam2_TrNum12_Comp_KL001B-KL001Y.mp4',
 '043024_Cam2_TrNum10_Comp_KL001B-KL001Y.mp4',
 '050324_Cam4_TrNum7_Comp_KL001B-KL001Y.mp4',
 '050324_Cam4_TrNum12_Coop_KL001B-KL001Y.mp4',
 '050324_Cam4_TrNum5_Coop_KL001B-KL001Y.mp4',
 '050324_Cam4_TrNum10_Comp_KL001B-KL001Y.mp4',
 '050524_Cam2_TrNum10_In